1.	Python 
1)	Написать на python генератор первых n чисел фибоначи.


In [ ]:
def fibonacci_generator(n):
    fibonacci = [0, 1]
    for i in range(0,n):
        fibonacci.append(fibonacci[i] + fibonacci[i+1])
    return fibonacci

2.	Написать Sql-запросы
1)	Вывести отдел с наибольшим числом сотрудников;
2)	Вывести список сотрудников, получающих заработную плату выше, чем у руководителя;


In [ ]:
%%sql sqlite://
SELECT 
    d.name AS department_name,
    COUNT(p.id) AS employee_count
FROM 
    Department d
JOIN 
    Personal p ON d.id = p.id_dep
GROUP BY 
    d.id, d.name
ORDER BY 
    employee_count DESC
LIMIT 1;

In [ ]:
%%sql sqlite://
SELECT 
    emp.id AS employee_id,
    emp.name AS employee_name,
    emp.sal AS employee_salary,
    mgr.name AS manager_name,
    mgr.sal AS manager_salary
FROM 
    Personal emp
JOIN 
    Personal mgr ON emp.id_head = mgr.id
WHERE 
    emp.sal > mgr.sal;

3.	Задачка на определение отрасли компании

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from lightgbm import LGBMClassifier

#Загрузим данные
pays = pd.read_csv('pays.csv')
inn_info = pd.read_csv('inn_info_public.csv')

#Объединеним данные
pays_agg = pays.groupby('hash_inn_kt').agg({
    'count': ['mean', 'sum'], 
    'sum': ['mean', 'sum'],
    'week': 'nunique'
}).reset_index()
pays_agg.columns = ['hash_inn'] + [f'feature_{i}' for i in range(len(pays_agg.columns)-1)]
data = pd.merge(inn_info, pays_agg, how='left', on='hash_inn')

#Обработаем целевую переменную
data_train = data[data['okved2'] != -1]
data_test = data[data['okved2'] == -1]

#Разделим данные на X и y
X = data_train.drop(columns=['okved2', 'hash_inn'])
y = data_train['okved2']
X_test = data_test.drop(columns=['okved2', 'hash_inn'])

#Обученим модели
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)
model = LGBMClassifier()
model.fit(X_train, y_train)

#Оценим модель
y_pred = model.predict(X_val)

#Предсказание модели
data_test['y'] = model.predict(X_test)
data_test['okved2_prob'] = model.predict_proba(X_test).max(axis=1)

#Отсортируем данные по столбцу 'hash_inn' в порядке возрастания и сохраним
sorted_data_test = data_test[['hash_inn', 'y']].sort_values(by='hash_inn', ascending=True)
sorted_data_test.to_csv('BogdanNeverov-18112024-SberIndustry.csv', index=False)

#Отсортируем данные по столбцу 'hash_inn' в порядке возрастания и сохраним
submission = data_test[['hash_inn', 'y', 'okved2_prob']].sort_values(by='hash_inn') 
submission.to_csv('submission-BogdanNeverov-18112024-SberIndustry.csv', index=False)